In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow-gpu

     |████████████████████████████████| 394.3MB 40kB/s 


In [15]:
import numpy as np
import cv2
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [16]:
train_path = "/content/drive/MyDrive/Facial Detection/train"
test_path = "/content/drive/MyDrive/Facial Detection/test"

In [10]:
classes = ['0','1','2','3','4','5','6'] 

In [11]:
## Read images and converting them to size 224,224,3

In [18]:
img_size = [224,224]

In [19]:
# Here we will use imagenet weights
inception = InceptionV3(input_shape=img_size + [3], weights='imagenet', include_top=False)

87916544/87910968 [==============================] - 1s 0us/step


In [20]:
# don't train existing weights
for layer in inception.layers:
    layer.trainable = False

In [21]:
#for getting number of output classes
folders = glob('/content/drive/MyDrive/Facial Detection/train/*')

In [22]:
# layers
x = Flatten()(inception.output)

In [23]:
prediction = Dense(len(folders), activation='softmax')(x)

# creating a model object
model = Model(inputs=inception.input, outputs=prediction)

In [24]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [25]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [26]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#generating the data
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [29]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Facial Detection/train',
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

Found 28719 images belonging to 7 classes.


In [30]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Facial Detection/test',
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')

Found 7178 images belonging to 7 classes.


In [70]:
result = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
1795/1795 [==============================] - 360s 200ms/step - loss: 7.1380 - accuracy: 0.5361 - val_loss: 9.4357 - val_accuracy: 0.4507
Epoch 2/20
1795/1795 [==============================] - 358s 200ms/step - loss: 7.3860 - accuracy: 0.5334 - val_loss: 8.7626 - val_accuracy: 0.4944
Epoch 3/20
1795/1795 [==============================] - 359s 200ms/step - loss: 7.1943 - accuracy: 0.5455 - val_loss: 9.9974 - val_accuracy: 0.4681
Epoch 4/20
1795/1795 [==============================] - 368s 205ms/step - loss: 7.1747 - accuracy: 0.5469 - val_loss: 8.8475 - val_accuracy: 0.4811
Epoch 5/20
1795/1795 [==============================] - 370s 206ms/step - loss: 7.0546 - accuracy: 0.5552 - val_loss: 8.9397 - val_accuracy: 0.5046
Epoch 6/20
1795/1795 [==============================] - 369s 206ms/step - loss: 7.2411 - accuracy: 0.5513 - val_loss: 8.8359 - val_accuracy: 0.5006
Epoch 7/20
1795/1795 [==============================] - 371s 207ms/step - loss: 7.1954 - accuracy: 0.5584 - val_

In [71]:
model.save('model_face_emotions_detection2.h5')